Normalizing data refers to the process of rescaling numerical features to a standard range, typically between 0 and 1. This is achieved using techniques like Min-Max scaling, where each value is subtracted by the minimum value and then divided by the range (max - min). Normalization is crucial when features have different units or scales (e.g., price in thousands and area in square feet), as models may give more importance to larger-scale features. It helps algorithms like gradient descent converge faster, improves model performance, and ensures that no feature dominates due to its scale.

Scaling data is essential because it ensures that all features contribute equally to a machine learning model. Features with different scales (e.g., one feature in thousands and another in fractions) can bias algorithms, especially those sensitive to the magnitude of values, like distance-based models (e.g., k-nearest neighbors, SVM) and gradient-based models. Without scaling, large-scale features may dominate the learning process, leading to poor model performance. Scaling techniques, such as Min-Max scaling or Standardization (Z-score normalization), bring all features into a comparable range or distribution, improving model accuracy, convergence speed, and stability during training.

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from datetime import datetime

# Load the dataset (assuming you've already downloaded it)
df = pd.read_csv("AmesHousing.csv")

Handling Missing Values with Median Imputation
Missing values are imputed by the median of the neighborhood group to preserve the local data distribution. This ensures that the imputed values are representative of their specific context, reducing bias compared to using a global mean.

In [4]:
# Example of filling LotFrontage with median value per neighborhood
df['Lot Frontage'] = df.groupby('Neighborhood')['Lot Frontage'].transform(lambda x: x.fillna(x.median()))

Encoding Categorical Variables
One-hot encoding transforms categorical variables (like 'MS Zoning') into binary columns, where each column represents a category. This allows the model to interpret categorical data as separate features without imposing any ordinal relationship.

In [6]:
# One-hot encode the 'MS Zoning' column
df = pd.get_dummies(df, columns=['MS Zoning'], drop_first=True)


Label Encoding for Ordinal Data
Label encoding assigns a numerical value to each category based on an inherent order. For example, 'Exter Qual' can be encoded using a numeric scale that reflects the quality levels, which can help models learn the ordinal relationship between categories.

In [7]:
# Label encode 'Exter Qual' as it has an order of quality levels
quality_order = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
df['Exter Qual'] = df['Exter Qual'].map(quality_order)

Binning Year Built into Decades
This technique categorizes the 'Year Built' feature into decades. Grouping years in a categorical manner helps capture historical trends and reduces granularity, potentially improving model performance by grouping similar properties together.

In [8]:
# Create a new column 'Decade Built' based on 'Year Built'
df['Decade Built'] = (df['Year Built'] // 10) * 10

Creating Interaction Features
Interaction features are created by combining two or more variables. For example, multiplying 'Gr Liv Area' with 'Overall Qual' creates a feature that represents the interaction between living area size and quality, which may reveal new insights for prediction.

In [9]:
# Create an interaction feature between 'Gr Liv Area' and 'Overall Qual'
df['LivingArea_Quality'] = df['Gr Liv Area'] * df['Overall Qual']


Log Transformation for Skewed Features
Log transformations help normalize highly skewed data. Applying this transformation to 'Lot Area' reduces skewness, making the data more normally distributed, which often improves model performance for algorithms sensitive to non-normal distributions.

In [10]:
# Apply log transformation to 'Lot Area' to reduce skewness
df['Lot Area Log'] = np.log1p(df['Lot Area'])


Polynomial features are higher-order terms created from existing features. Squaring or cubing a feature like 'Overall Qual' allows the model to capture non-linear relationships that linear models might miss, enhancing predictive power.

In [11]:
# Create squared and cubic terms for 'Overall Qual'
df['Overall Qual Sq'] = df['Overall Qual'] ** 2
df['Overall Qual Cub'] = df['Overall Qual'] ** 3


By summing related features like '1st Flr SF', '2nd Flr SF', and 'Garage Area', you can create a new feature that provides the total area of a house. This captures the overall size, potentially improving model understanding of house value.

In [12]:
# Calculate total area including basement and garage areas
df['Total Area'] = df['1st Flr SF'] + df['2nd Flr SF'] + df['Total Bsmt SF'] + df['Garage Area']


The 'Sale Month' and 'Sale Year' features are extracted from date columns. These time-related features allow the model to learn seasonal and temporal patterns that may impact housing prices, such as changes in the market over time.



In [13]:
# Extract month and year from sale date columns
df['Sale Month'] = df['Mo Sold']
df['Sale Year'] = df['Yr Sold']


A bathroom-to-bedroom ratio is created by dividing the total number of bathrooms by the number of bedrooms. This ratio can help capture the relative spaciousness and functionality of a home, which could be a strong predictor of sale price.

In [14]:
# Create a bathroom-to-bedroom ratio
df['Bath_to_Bed_Ratio'] = (df['Full Bath'] + 0.5 * df['Half Bath']) / (df['Bedroom AbvGr'] + 1)


In [19]:
df = pd.read_csv("AmesHousing.csv")
df['Neighborhood']

0         NAmes
1         NAmes
2         NAmes
3         NAmes
4       Gilbert
         ...   
2925    Mitchel
2926    Mitchel
2927    Mitchel
2928    Mitchel
2929    Mitchel
Name: Neighborhood, Length: 2930, dtype: object

Target encoding replaces categories in a feature with the mean target value (e.g., 'SalePrice') for each category. For 'Neighborhood', encoding replaces it with the average sale price, capturing neighborhood-level price trends and helping improve predictive performance.



In [20]:
# Encode 'Neighborhood' by the mean 'SalePrice' in each neighborhood
df['Neighborhood_Target'] = df.groupby('Neighborhood')['SalePrice'].transform('mean')


KMeans clustering groups neighborhoods based on the mean sale price. This technique creates a new feature that identifies neighborhood clusters with similar price characteristics, which could help the model better understand how neighborhoods influence sale price.

In [21]:
# Cluster neighborhoods based on mean 'SalePrice' using KMeans
kmeans = KMeans(n_clusters=5, random_state=0)
df['Neighborhood Cluster'] = kmeans.fit_predict(df[['Neighborhood_Target']])


Frequency encoding replaces categories with their count of occurrences in the dataset. This approach is useful when categorical features have many levels, as it provides a numeric value that reflects the commonality of each category within the dataset.

In [22]:
# Frequency encode 'Neighborhood'
df['Neighborhood_Freq'] = df['Neighborhood'].map(df['Neighborhood'].value_counts())


By summing up features like 'TotRms AbvGrd', 'Bedroom AbvGr', and 'Kitchen AbvGr', we create a new feature that represents the total number of rooms. This helps the model understand the overall scale and livability of the house.

In [ ]:

# Count total rooms above ground (living area)
df['Total Rooms'] = df['TotRms AbvGrd'] + df['Bedroom AbvGr'] + df['Kitchen AbvGr']


An indicator variable is created to flag extreme values, such as houses with large living areas. This allows the model to treat extreme cases differently, which could help in understanding the impact of large homes on sale prices.

In [ ]:
# Indicator for large 'Gr Liv Area' values (e.g., above 4000 sqft)
df['Large Living Area'] = (df['Gr Liv Area'] > 4000).astype(int)


The ratio of floor areas (e.g., '1st Flr SF' to '2nd Flr SF') is calculated and transformed into its logarithmic form. This helps capture relative size differences between floors in a way that is less sensitive to large scale differences.

In [ ]:
# Calculate log-ratio of '1st Flr SF' to '2nd Flr SF'
df['Floor SF Log Ratio'] = np.log1p(df['1st Flr SF'] / (df['2nd Flr SF'] + 1))


Normalization rescales numeric features into a specific range, such as 0 to 1, using MinMaxScaler. This ensures that features like 'Lot Area' are on the same scale, which is important for models sensitive to the magnitude of values.



In [ ]:
# Normalize 'Lot Area' using MinMaxScaler
scaler = MinMaxScaler()
df['Lot Area Scaled'] = scaler.fit_transform(df[['Lot Area']])


A flag variable is a binary indicator (0 or 1) created to signify the presence or absence of a specific condition, such as having a pool. This helps the model easily differentiate between homes with and without certain features.

In [ ]:
# Create a flag for houses with a pool
df['Has Pool'] = (df['Pool Area'] > 0).astype(int)

This feature compares a house's 'Lot Area' to the average lot area in the neighborhood. The ratio helps the model understand how a house's size relates to others in the same area, capturing potential market dynamics.

In [ ]:
# Create a feature comparing lot area to the neighborhood average
df['Lot Area to Neighborhood Avg'] = df['Lot Area'] a/ df.groupby('Neighborhood')['Lot Area'].transform('mean')


Box-Cox transformation normalizes skewed data by applying a power transformation. It is particularly useful for highly skewed features like 'Gr Liv Area'. The transformation helps improve model assumptions, such as normality, for algorithms that require it.

In [ ]:
# Apply Box-Cox transformation to 'Gr Liv Area' to normalize distribution
df['Gr Liv Area BoxCox'], _ = boxcox(df['Gr Liv Area'] + 1)  # Add 1 to handle zero values


The Box-Cox transformation is a statistical technique used to stabilize variance and make data more normally distributed. he Box-Cox transformation is useful for handling skewed data and reducing heteroscedasticity, making it easier for models to learn patterns.